In [ ]:
!pip install segmentation_models_pytorch
!pip install -q -U albumentations
!pip install timm

  Cloning https://github.com/zhanghang1989/PyTorch-Encoding.git to /tmp/pip-req-build-slhdewra
  Running command git clone -q https://github.com/zhanghang1989/PyTorch-Encoding.git /tmp/pip-req-build-slhdewra
ERROR: Operation cancelled by user


In [ ]:
import segmentation_models_pytorch as smp
from tqdm import tqdm
import gc

import math
from torch.optim.optimizer import Optimizer, required


from sklearn.model_selection import GroupKFold, KFold
import torch
from torch import nn
import torchvision
import cv2
import os
import numpy as np
import pandas as pd

from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts, _LRScheduler
from scipy.ndimage.interpolation import zoom
import albumentations as A
from torch.nn import functional as F
from albumentations.pytorch import ToTensorV2

from pycocotools.coco import COCO

import matplotlib.pyplot as plt
import sys
import time
import random
import timm

import zipfile

In [ ]:
CFG = {
    "encoder": "timm-efficientnet-b4"
    "mean": (0.485, 0.456, 0.406),
    "std": (0.229, 0.224, 0.225)
}

In [1]:
device = "cuda"
submission = pd.read_csv('/content/drive/MyDrive/trash_segmentation/data/sample_submission.csv', index_col=None)
test_path = '/content/drive/MyDrive/trash_segmentation/data/test.json'
MODEL_PATHS = ['/content/drive/MyDrive/trash_segmentation/models/timm-efficientnet-b4_0.pth']
SUBMISSION_PATH = "/content/drive/MyDrive/trash_segmentation/data/submission.csv"
OUT_MASKS = f'/content/drive/MyDrive/trash_segmentation/masks.zip'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join('/content/drive/MyDrive/trash_segmentation/data', image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # Unknown = 1, General trash = 2, ... , Cigarette = 11
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)
            masks = masks

            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
def get_validation_augmentations():
    return A.Compose([
        A.Normalize(mean=CFG['mean'], std=CFG['std'], max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ],p=1.0)

In [ ]:
test_ds = CustomDataLoader(test_path, mode="test", transform=get_validation_augmentations())

test_loader = DataLoader(test_ds, 
                          batch_size=1, 
                          num_workers=4,
                         collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
def test(models, data_loader, device):
  
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    file_name_list = []
    outs = None
    preds_array = np.empty((0, size*size), dtype=np.long)
    pbar = tqdm(enumerate(data_loader), total=len(data_loader), position=0, leave=True)
    with torch.no_grad():
      with zipfile.ZipFile(OUT_MASKS, 'w') as mask_out:
        for step, (imgs, image_infos) in pbar:
            # print(imgs)
            # print(imgs)
            imgs = torch.stack(imgs)
            for model in models:
                # outs = model(imgs.to(device).float())
                if outs == None:
                    outs = model(imgs.to(device).float())
                else:
                    outs += model(imgs.to(device).float())

            # inference (512 x 512)
            
            flips = [[-1],[-2],[-2,-1]]
            for flip in flips:
                flip_img = torch.flip(imgs, flip)
                for model in models:
                    out = model(flip_img.to(device).float())
                    out = torch.flip(out, flip)
                    outs += out

            oms = torch.argmax(outs, dim=1).detach().cpu().numpy()
            plt.imshow(np.squeeze(oms))
            # print(oms.shape)
            # break
            file_name = image_infos[0]['file_name'].split("/")
            file_name[0] += "_masks"
            file_name[1] = file_name[1][:-4]
            file_name = "/".join(file_name)

            m = cv2.imencode(".png", oms.squeeze())[1]
            mask_out.writestr(f"{file_name}.png", m)
            
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                # print(mask.shape)
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)

            oms = np.array(temp_mask)
            # print(oms.shape)
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            # print(oms.shape)
            # return
            preds_array = np.vstack((preds_array, oms))
            
            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    return file_names, preds_array

In [ ]:
models = []

In [ ]:
for path in MODEL_PATHS:
    model = smp.DeepLabV3Plus(CFG["encoder"], encoder_weights=None, in_channels=3, classes=12).to(device)
    checkpoint = torch.load(path, map_location=device)
    # model.load_state_dict(checkpoint['model'])
    model.load_state_dict(checkpoint)
    model.eval()
    model.to(device)
    models.append(model)

In [ ]:
# test set에 대한 prediction
file_names, preds = test(models, test_loader, device)

Start prediction.


100%|██████████| 837/837 [05:08<00:00,  2.71it/s]

End prediction.


In [ ]:
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(SUBMISSION_PATH, index=False)